In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import seaborn as sns
import math
from data_gen import gen_df

def fig4_J():
    ## Importing data and making new dataframe
    filename = 'imdb.csv'
    df = gen_df(filename)
    imdb = df.copy()
    
    ## Making variable that shows total numbers of award win per decade
    imdb['nrOfWins_tot'] = imdb.groupby('decade')['nrOfWins'].transform(lambda x : x.sum())
    
    ## Generating variable that take a movies share of total win in the decade
    imdb['nrOfWins_share'] = imdb['nrOfWins']/imdb['nrOfWins_tot']*100
    
    ## Taking the mean/median/P75 on the win share in each decade
    imdb['nrOfWins_share_avg'] = imdb.groupby('decade')['nrOfWins_share'].transform(lambda x: x.mean())
    imdb['nrOfWins_share_med'] = imdb.groupby('decade')['nrOfWins_share'].transform(lambda x: x.median())
    imdb['nrOfWins_share_p75'] = imdb.groupby('decade')['nrOfWins_share'].transform(lambda x: np.percentile(x,95))


    ## Generate variable that counts number of observation/movies in each year
    imdb['temp']=1
    imdb['tot_film'] = imdb.groupby('decade')['temp'].transform(lambda x : x.sum())
    del imdb['temp']

    ## Creating a new dataframe that have one observation per decade
    y = pd.DataFrame(imdb.groupby('decade')['nrOfWins_tot'].first())
    y['nrOfWins_share_avg'] = imdb.groupby('decade')['nrOfWins_share_avg'].first()
    y['nrOfWins_share_med'] = imdb.groupby('decade')['nrOfWins_share_med'].first()
    y['nrOfWins_share_p75'] = imdb.groupby('decade')['nrOfWins_share_p75'].first()
    y['tot_film'] = imdb.groupby('decade')['tot_film'].first()
    y.reset_index(inplace=True)

    ## Plotting the total numbers of wins and movies and the average share of wins per decade
    def fig(temp1, temp2):
        fig = plt.figure(figsize=(14,6))

        # Figure 1
        ax1 = fig.add_subplot(1,2,1)
        ax1.plot(y['decade'], y['nrOfWins_share_avg'], color='blue', label='Average share of wins')
        if temp1 == True:
            ax1.plot(y['decade'], y['nrOfWins_share_med'], color='blue', label='Median share of wins', linestyle='dashed')
        ax1.set_ylim([0,0.7])
        if temp2 == True:
            ax1.plot(y['decade'], y['nrOfWins_share_p75'], color='blue', label='P75 share of wins', linestyle='dotted')
            ax1.set_ylim([0,4])
        ax1.legend(loc='upper right');
        ax1.set_ylabel('Share of total award wins in the decade, pct.')
        ax1.set_title(f'Average numbers of award wins normalized by total award wins')        
        
        # Figure 2
        ax2 = fig.add_subplot(1,2,2)
        ax2.plot(y['decade'], y['tot_film'], color='blue', label='Number of movies')
        ax2.plot(y['decade'], y['nrOfWins_tot'], color='black', label='Numbers of award wins')
        ax2.legend(loc='upper left');
        ax2.set_ylabel('Numbers of movies/award wins')
        ax2.set_title(f'Numbers of movies and award wins by decade, 1920-2013')

    
        # Interactive part: Making checkbox stat-types
    widgets.interact(fig,
        temp1=widgets.Checkbox(description='Median', value=True, disabled=False),
        temp2=widgets.Checkbox(description='P75', value=True, disabled=False),
    );
fig4_J()

interactive(children=(Checkbox(value=True, description='Median'), Checkbox(value=True, description='P75'), Out…